In [101]:
import pickle
import simpleaudio as sa
from keyword_spotting.feature_extraction.utils import read_wav
import numpy as np
from scipy.io.wavfile import write
import pyaudio
import wave

In [133]:
filename = "lolo_silence1.wav"

In [134]:
chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1
fs = 16000  # Record at 44100 samples per second
seconds = 2

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [135]:
wave_obj = sa.WaveObject.from_wave_file(filename)
play_obj = wave_obj.play()
play_obj.wait_done()  # Wait until sound has finished playing

# Predecir con el modelo cual es el label

In [6]:
import tensorflow as tf
from keyword_spotting.model import models

In [34]:
configFILE='/Users/emi/cnn_residual2_19_04_14_Jun_13_2021_results.pkl'
modelFILE = '/Users/emi/cnn_residual2_19_04_14_Jun_13_2021'
with open(configFILE, 'rb') as f:
    config, time, predictions = pickle.load(f)
config

{'model': {'name': 'cnn_residual2',
  'params': {'n_residuals': 3, 'n_filters': 19, 'learning_rate': 0.01},
  'windowed': False},
 'train': {'batch_size': 64, 'epochs': 27, 'reduce_on_plateau': True}}

In [35]:
input_shape = [100, 40]
number_of_classes=12

model = models[config["model"]["name"]](input_shape, number_of_classes, **config["model"]["params"])
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100, 40)]    0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 100, 40, 1)   0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 98, 38, 19)   190         lambda_4[0][0]                   
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 49, 19, 19)   0           conv2d_7[0][0]                   
____________________________________________________________________________________________

In [36]:
# Restore the weights
model.load_weights(f'{modelFILE}')

In [51]:
from keyword_spotting.feature_extraction.utils import extract_features, read_wav, windowed
import numpy as np
from keyword_spotting.predictions import labels

In [59]:
labels

['yes',
 'no',
 'up',
 'down',
 'left',
 'right',
 'on',
 'off',
 'stop',
 'go',
 'unknown',
 'silence']

# Predict the label using one of our models

In [136]:
sample_rate, signal = read_wav(filename)
data = extract_features(signal, sample_rate)
data = np.expand_dims(data[:100,:], axis=0)
predictions = model.predict(data)

In [137]:
predictions

array([[6.39143982e-04, 1.05054794e-04, 5.44560608e-04, 4.92981890e-05,
        1.27227577e-05, 1.04601559e-05, 1.79339841e-04, 2.25342766e-04,
        1.24152109e-04, 8.45095856e-05, 6.10138741e-05, 9.97964382e-01]],
      dtype=float32)

In [138]:
label_predicted = np.argmax(predictions)
labels[label_predicted]

'silence'